<h2>Table comparing different possible Cl2-to-CH4 ratios</h2>

In [2]:
# Some imports to stop some annoying yellow underlines
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
import importlib
import types

# Very important lines to run the setup notebooks!!!
%run do_plot_setup.ipynb
%run do_radical_count_setup.ipynb
get_cl_radical_budget = get_cl_radical_budget # Tautology to get rid of the yellow underlines
import cl2_utilities as cl2_utils

# Extract some shared values that are used to facilitate plotting. Call 'display' on any of them to see their contents.
plot_default_properties = get_and_apply_plot_defaults() #Ignore yellow underline
line_default_properties = get_default_line_properties() #Ignore yellow underline
cl2_experiment_constants = get_cl2_experiment_constants() #Ignore yellow underline
(all_conversion_data,all_bypass_data) = get_experimental_data() #Ignore underline. These two dataframes have all the experimental results.

In [3]:
# Load the data from the Cl2 sweep to use here

# Load data for the Cl2 sweep trial
current_data = types.SimpleNamespace()

# Select data from only the the experiments we want
exps = ["09-28-24 Vary Cl2 (20-90 ppm) at Fixed CH4","09-19-24 Vary Cl2 (5-30 ppm) at Fixed CH4"]
bypass_data = all_bypass_data[all_bypass_data["experiment_name"].isin(exps)]
conversion_data = all_conversion_data[all_conversion_data["experiment_name"].isin(exps)]
conversion_data = conversion_data[(conversion_data['experiment_name']=="09-19-24 Vary Cl2 (5-30 ppm) at Fixed CH4") | # Use exp. with 100 ppm Cl2 tank for Cl2 5-30 ppm and exp. with 1000 ppm Cl2 tank for 45-90 ppm.
                                  (conversion_data['Cl2 LabJack: Cl2 reading minus zero (mV) baseline']>40)] # Proxy for >=45 ppm is a Cl2 measurement of more than 85mV
conversion_data=conversion_data.sort_values(by='Cl2 LabJack: Cl2 reading minus zero (mV) baseline',ascending=True)
current_data.conversion_data = conversion_data
current_data.bypass_data = bypass_data

# The Cl2 tank ppms are set manually here based on notes taken during each experiment; it's not recorded automatically in software.
cl2_tank_ppms = np.array([1000 if b>40 else 100 for b in conversion_data['Cl2 LabJack: Cl2 reading minus zero (mV) baseline']]) # We've confirmed that in this subset of trials, only Cl2 conc's 45 and over used the 1000 ppm Cl2 tank

# Extract the values and 95% CI from the CH4 conversion data
current_data.ch4_baselines = conversion_data["Selected: CH4 (ppm) baseline"]
((current_data.ch4_conversions,current_data.ch4_95_ci),) = cl2_utils.extract_spectrometer_data_from_conversions(conversion_data,
            fields=['Selected: CH4 (ppm)'], percent_accuracy_95 = cl2_experiment_constants.picarro_percent_accuracy_95, absolute_accuracy_95 = cl2_experiment_constants.picarro_absolute_accuracy_95)

# Extract the values and 95% CI from the Cl2 conversion data
(current_data.cl2_baseline, current_data.cl2_conversions, current_data.cl2_conversion_95_ci) = cl2_utils.extract_cl2_data_from_conversions(conversion_data,bypass_data,cl2_tank_ppms,cl2_experiment_constants.cl2_mfc_sccm_accuracy_95,cl2_experiment_constants.cl2_node_absolute_accuracy_95)
current_data.inlet_cl2 = current_data.cl2_baseline

# Compute the Cl* efficiencies
(current_data.cl_budget,current_data.cl_budget_95_ci) = get_cl_radical_budget(current_data.inlet_cl2)
current_data.cl_per_ch4 = current_data.cl_budget / current_data.ch4_conversions
current_data.cl2_per_ch4_95_ci = cl2_utils.get_95_ci_of_ratio(current_data.cl_budget,current_data.cl_budget_95_ci,current_data.ch4_conversions,current_data.ch4_95_ci)

# Save to variable and report success
cl2_sweep_data = current_data
print("Loaded results for plotting into object 'cl2_sweep_data'.")

Loaded results for plotting into object 'cl2_sweep_data'.


In [27]:
# Imports
from IPython.display import display, HTML
from pretty_html_table import build_table

# Tabulate the non-cost values that come straight from the experiments
inlet_CH4 = 30 # ppm
df = pd.DataFrame()
df['Inlet [Cl2] (ppm)'] = np.array(cl2_sweep_data.cl2_baseline)
df['Inlet Cl2-to-CH4 ratio (–)'] = np.array(cl2_sweep_data.cl2_baseline)/inlet_CH4
df['CH4 conversion (ppm)'] = np.array(cl2_sweep_data.ch4_conversions)
df['CH4 conversion (%)'] = 100*np.array(cl2_sweep_data.ch4_conversions)/inlet_CH4
df['CH4 converted per Cl2 added ($\\rm \\eta_{\\rm Cl_2}$, –)'] = np.array(cl2_sweep_data.ch4_conversions)/np.array(cl2_sweep_data.cl2_baseline)
df['CH4 converted per Cl· produced ($\\rm \\eta_{\\rm Cl^\\bullet}$, –)'] = 1.0/np.array(cl2_sweep_data.cl_per_ch4)

# Tabulate the chlorine cost
#tons_cl2_per_ton_ch4 = (70.9060/16.04)*df['Cl2 added per CH4 converted (–)'] # (g-cl2/mol-cl2)*(mol-ch4/g-ch4)*(mol_cl2/mol_ch4) = (g-cl2/g-ch4)
df['Cl2 cost per ton CH4 ($)'] = (1.0/df['CH4 converted per Cl2 added ($\\rm \\eta_{\\rm Cl_2}$, –)'])*cl2_experiment_constants.cost_cl2_usage_coefficient

#moles_photons_per_ton_ch4 = 62500*df['Cl· Usage (–)']*(1.0/cl2_experiment_constants.reflector_efficiency)*0.5 #Each UV photon produces 2 Cl*'s when it hits a Cl2
df['Lighting cost per ton CH4 ($)'] = (1.0/df['CH4 converted per Cl· produced ($\\rm \\eta_{\\rm Cl^\\bullet}$, –)'])*cl2_experiment_constants.cost_cl_radical_usage_coefficient
# And get the combined cost
df['Lighting and Cl2 cost per ton CH4 ($)'] = df['Lighting cost per ton CH4 ($)']+df['Cl2 cost per ton CH4 ($)']

# Apply the significant figure calculation
from math import floor, log10
for col in df.columns:
    df[col]=df[col].apply(lambda x: (round(x, 1 - int(floor(log10(abs(x)))))))

# Machinery to add subscripts on CH4 and Cl2
def do_subscripts(text):
    text = text.replace('CH4','CH<sub>4</sub>')
    text = text.replace('Cl2','Cl<sub>2</sub>')
    return text

# Generate the actual table. Annoying workaround needed to print 2 sig figs for 1-digit integers, i.e. '3.0' instead of just '3'.
from IPython.display import display, Markdown, Latex
md_text = (do_subscripts(df.to_markdown(intfmt=",",numalign='center', stralign='center', index=False)))
for i in '123456789':
    md_text = md_text.replace(('           '+i+'            '),('           '+i+'.0            '))
display(Markdown(md_text))

# Save the table as a CSV file
df.to_csv('exported_images/cl2_ch4_comparison_table.csv', index=False)


|  Inlet [Cl<sub>2</sub>] (ppm)  |  Inlet Cl<sub>2</sub>-to-CH<sub>4</sub> ratio (–)  |  CH<sub>4</sub> conversion (ppm)  |  CH<sub>4</sub> conversion (%)  |  CH<sub>4</sub> converted per Cl<sub>2</sub> added ($\rm \eta_{\rm Cl_2}$, –)  |  CH<sub>4</sub> converted per Cl· produced ($\rm \eta_{\rm Cl^\bullet}$, –)  |  Cl<sub>2</sub> cost per ton CH<sub>4</sub> ($)  |  Lighting cost per ton CH<sub>4</sub> ($)  |  Lighting and Cl<sub>2</sub> cost per ton CH<sub>4</sub> ($)  |
|:-------------------:|:----------------------------:|:----------------------:|:--------------------:|:--------------------------------------------------------:|:-----------------------------------------------------------------:|:--------------------------:|:-------------------------------:|:---------------------------------------:|
|         10          |             0.33             |          5.7           |          19          |                           0.57                           |                               0.36                                |            1800            |              2300               |                  4200                   |
|         20          |             0.67             |           12           |          41          |                           0.62                           |                               0.36                                |            1700            |              2300               |                  4000                   |
|         30          |              1.0               |           19           |          62          |                           0.62                           |                               0.36                                |            1700            |              2300               |                  4000                   |
|         45          |             1.5              |           23           |          77          |                           0.51                           |                                0.3                                |            2000            |              2800               |                  4900                   |
|         60          |              2.0               |           28           |          94          |                           0.47                           |                               0.27                                |            2200            |              3100               |                  5300                   |
|         90          |              3.0               |           30           |         100          |                           0.33                           |                               0.19                                |            3100            |              4300               |                  7500                   |